In [1]:
import pandas as pd
#from dask.distributed import Client, LocalCluster
#import dask.dataframe as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import json
from pathlib import Path
from operator import itemgetter
import os

from sklearn.feature_extraction.text import CountVectorizer, BaseEstimator, ENGLISH_STOP_WORDS, TfidfTransformer

In [ ]:
#client = Client()
#cluster = LocalCluster()

In [ ]:
def json_reader(absolute_parent_path):
    total_data = []
    file_list = (str(i) for i in Path('/data/capstone_data/messages/inbox/').glob('*/*.json'))
    for file in file_list:
        with open(file) as json_data:
            try:
                yield json.load(json_data)
                #total_data.append(data_raw)
                
            except:
                pass
    return total_data

In [ ]:
inbox_data = json_reader('/data/capstone_data/messages/inbox/')

In [ ]:
archive_data = json_reader('/data/capstone_data/messages/archived_threads//')

In [ ]:
all_data = [inbox_data, archive_data]

In [ ]:
def unpack_list(dict_item):
    content_list = []
    for entry in dict_item:
        for i in range(len(entry['messages'])):
            try:
                content = pd.DataFrame([i for i in entry['messages']][i])
                content_list.append(content)
            except:
                pass
    return pd.concat(content_list)

In [ ]:
#all_comments = unpack_list(i) for i in all_data

In [ ]:
%time
all_comments.to_csv('fb_chat_history_sept_2009_oct_2020_test.csv')

In [2]:
all_comments = pd.read_csv('fb_chat_history_sept_2009_oct_2020.csv')

/home/vivi/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
all_comments['sender_name'].value_counts()

Vivian Nguyen                89998
Zee Ng                       53112
Jane Yuan                    44814
Chieko K. Kagiyama           42096
Aly P. Lee                   23786
                             ...  
Jayne Elizabeth McFarland        2
Rachel Williamson                2
Monica Tolete                    2
Alisa Phung                      2
Clint Finley                     2
Name: sender_name, Length: 1174, dtype: int64

In [ ]:
len(all_comments)

In [ ]:
all_comments.tail()

In [ ]:
all_comments['sender_name'].value_counts().tail(20)

In [ ]:
msno.matrix(all_comments)

In [ ]:
all_comments['timestamp_ms'].min(), all_comments['timestamp_ms'].max()

In [ ]:
all_messages_no_context = all_comments['content'].dropna()

In [ ]:
all_messages_no_context

In [ ]:
cv = CountVectorizer()
matrix = cv.fit_transform(all_messages_no_context)
freqs = zip(cv.get_feature_names(), matrix.sum(axis=0).tolist()[0])
df_words_no_stop = pd.DataFrame(freqs, columns=['word','count'])
top_30_no_stop = df_words_no_stop.sort_values(by='count', ascending=False).head(30)

In [ ]:
my_stop_words = ['to', 'and', 'the', 'you', 'it', 'my', 'of', 'in', 'is', 'that', 'for', 'me', 'but', 'this', 'so', 'like', 'was', 'with', 'on', 'just',
             'be', 'have', 'not', 'we', 'if', 'she','are', 'can', 'up', 'at', 'her', 'out', 'as', 'your', 'they', 'about', 'do', 'or', 'all', 'when', 
             'what', 'one', 'also', 'get', 'know', 'from', 'an', 'all', 'how', 'now', 'who', 'then', 'too', 'more', 'there', 'because', 'really', 
             'want', 'them', 'will', 'would', 'am', 'been', 'people', 'think', 'got', 'feel', 'going', 'iâ', 've', 'itâ', 'donâ', 'time', 'youâ', 'll',
                'don', 'https', 'll', 'thatâ', 'im']

In [ ]:
stop_words = ENGLISH_STOP_WORDS.union(my_stop_words)

In [ ]:
cv = CountVectorizer(stop_words=stop_words)

In [ ]:
matrix = cv.fit_transform(all_messages_no_context)
freqs = zip(cv.get_feature_names(), matrix.sum(axis=0).tolist()[0])
df_words = pd.DataFrame(freqs, columns=['word','count'])

In [ ]:
top_30 = df_words.sort_values(by='count', ascending=False).head(30)

In [ ]:
top_30.head()

In [ ]:
plt.figure(figsize=(25,23), facecolor='white')
plt.subplot(2,1,1)
sns.barplot(x=top_30['word'], y=top_30['count'])
plt.title('Top 30 with Stop Words', fontsize=30)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.savefig('top_30_w_stop.png')
plt.subplot(2,1,2)
sns.barplot(x=top_30_no_stop['word'], y=top_30_no_stop['count'])
plt.gca().invert_yaxis()
plt.title('Top 30 no Stop Words', fontsize=30, pad=20)
plt.gca().xaxis.tick_top()
plt.xticks(fontsize=15)
plt.yticks(fontsize=15);
plt.savefig('top30_with_without.png')

In [ ]:
matrix

In [ ]:
ttran = TfidfTransformer(use_idf=True, smooth_idf=True)
df_tfdif_all = ttran.fit_transform(matrix)

In [ ]:
# top_words to dataframe sorted by highest occurance
most_freq_tf = pd.DataFrame(sorted(df_tfdif_all, key = lambda x: x[1], reverse = True))

In [ ]:
most_freq_tf